# What is my coverage in the city?

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as SF

from dotenv import load_dotenv
import os



load_dotenv()
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")


connection_parameters = {
  "account": SF_ACCOUNT,
  "user": SF_USER,
  "password": SF_PASSWORD,
}
new_session = Session.builder.configs(connection_parameters).create()

In [2]:
db = "PANUCCIS_PIZZA"
schema = "POS_MAIN"
WH = "PANUCCIS_PIZZA_WH"
table_stores = "STORES"
table_customers = "CUSTOMERS"
table_orders = "ORDERS_HEADER"


new_session.use_database(db)
new_session.use_schema(schema)
new_session.use_warehouse(WH)

For geospatial analytics, snowflake has a native `geometry` and `geography` object to allow for various geospatial operations. Fairly similar but there are some differencees. Depending upon the task at hand, you may choose to go one way or another. Certain geospatial functions are limited to one of the 2 variants. Another thing to keep in mind, is that the default UoM for these systems is based on the chosed SRID. eg: 4326 uses degrees and 3857 is meters


1. Geometry Data Type
    - The geometry data type is used for planar (flat, 2D) spatial data. It assumes a Cartesian coordinate system, where the Earth is treated as a flat plane.
    This data type is ideal for small-scale geospatial operations where the curvature of the Earth can be ignored, such as city planning, local mapping, or small-area analytics.
    - Operations using the geometry type are performed in a flat, Euclidean space, which makes calculations simpler and faster for small areas.
    Example Use Case:

    - Calculating distances between points in a city or determining whether a point lies within a polygon representing a neighborhood.

2.  Geography Data Type
    - The geography data type is used for geospatial data on a spherical model of the Earth. It accounts for the Earth's curvature and uses a geodetic coordinate system (latitude and longitude).
    - This data type is suitable for large-scale geospatial operations, such as global mapping, navigation, or analytics that span large distances.
    Operations using the geography type are performed on a spherical surface, which ensures accuracy over long distances.
    Example Use Case:

    - Calculating the shortest distance between two cities on different continents or determining whether a point lies within a country boundary.

In [31]:
# We'll  create a view that is downstream of stores data and includees the geospatial data for us
view_name = "STORES_W_GEOSPATIAL"

stores_view = new_session.sql(f"SELECT STORE_ID, LONGITUDE, LATITUDE from {db}.{schema}.{table_stores}")

# Create a new column for the geospatial representation of the location
stores_view = stores_view.with_column(
    "STORE_LOCATION_GEOGRAPHY", 
    SF.call_function("ST_MAKEPOINT",SF.col("LONGITUDE"),SF.col("LATITUDE"))
)

# Create a geometry column with a Mercator projection
stores_view = stores_view.with_column(
    "STORE_LOCATION_GEOMETRY",
        SF.call_function(
            "TO_GEOMETRY", 
            SF.col("STORE_LOCATION_GEOGRAPHY")
            )
)
stores_view = stores_view.drop(["LONGITUDE", "LATITUDE"])
# stores_view.show()

# Create a radius of 1 mile and 2 miles around each store



#### Explanation of Using ST_BUFFER for Drawing a Radius in Snowflake

The ST_BUFFER function in Snowflake is a geospatial function that creates a buffer (a circular or polygonal area) around a given geometry object. However, it is important to note that ST_BUFFER only works with geometry data types, which assume a flat, Cartesian coordinate system. This means that the function does not inherently account for the curvature of the Earth, and distances must be provided in the same units as the geometry's coordinate system

#### Base Data in Latitude and Longitude
In your case, the base data points (store locations) are in latitude and longitude, which are expressed in decimal degrees. Since ST_BUFFER operates on geometry data types, the radius argument must also be provided in decimal degrees, not in meters or miles. This requires converting the desired radius (e.g., 1 mile, 2 miles, or 5 miles) into decimal degrees.

#### Conversion of Miles to Decimal Degrees
The conversion from miles to decimal degrees depends on the location on Earth because the distance represented by one degree of latitude or longitude varies based on the Earth's curvature. For simplicity:

>Latitude:\
>One degree of latitude is approximately 69 miles everywhere on Earth.\
>Therefore, 1 mile ≈ 1 / 69 ≈ 0.0145 degrees of latitude.

>Longitude:\
>The distance represented by one degree of longitude varies with latitude. At the equator, 1 degree of longitude is approximately 69 miles, but this decreases as you move toward the poles.\
>In `Frisco, Texas` (latitude ~33°N), 1 degree of longitude is approximately 57 miles.\
>Therefore, 1 mile ≈ 1 / 57 ≈ 0.0176 degrees of longitude.

Average Conversion:

To simplify the calculation, you can use an average value for both latitude and longitude. For example:\
1 mile ≈ 0.01605 degrees (average of 0.0145 and 0.0176).

In [32]:
mile_list = [1, 2]
degree_list = [round(m*0.01605,5) for m in mile_list]

In [33]:
for m,d in zip(mile_list, degree_list):
    stores_view = stores_view.with_column(
        f"STORE_LOCATION_{m}MILE_DELIVERY_RADIUS", 
        SF.call_function(
            "ST_BUFFER", 
            SF.col("STORE_LOCATION_GEOMETRY"), 
            d
        )
    )
# stores_view.show()

In [34]:
# Save the view to Snowflake
stores_view.create_or_replace_view(
    f'{db}.{schema}.{view_name}')

[Row(status='View STORES_W_GEOSPATIAL successfully created.')]

# Use folium to visualize the delivery radius

In [35]:
query = f"""
    SELECT 
        STORE_ID, 
        ST_X(STORE_LOCATION_GEOMETRY) AS LONGITUDE, 
        ST_Y(STORE_LOCATION_GEOMETRY) AS LATITUDE,
        STORE_LOCATION_1MILE_DELIVERY_RADIUS, 
        STORE_LOCATION_2MILE_DELIVERY_RADIUS
    FROM {db}.{schema}.{view_name}
"""

In [36]:
import geopandas as gpd
from shapely.geometry import shape
df = new_session.sql(query).to_pandas()

gpd_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))

# Convert delivery radius columns to polygons
gpd_df['STORE_LOCATION_1MILE_DELIVERY_RADIUS'] = gpd_df['STORE_LOCATION_1MILE_DELIVERY_RADIUS'].apply(lambda x: shape(eval(x)))
gpd_df['STORE_LOCATION_2MILE_DELIVERY_RADIUS'] = gpd_df['STORE_LOCATION_2MILE_DELIVERY_RADIUS'].apply(lambda x: shape(eval(x)))

In [37]:
gpd_df

,STORE_ID,LONGITUDE,LATITUDE,STORE_LOCATION_1MILE_DELIVERY_RADIUS,STORE_LOCATION_2MILE_DELIVERY_RADIUS,geometry
0,3431,-96.87610,33.18667,"MULTIPOLYGON (((-96.86004999999999 33.18667, -...","MULTIPOLYGON (((-96.844 33.18667, -96.84461679...",POINT (-96.8761 33.18667)
1,4464,-96.79917,33.07242,"MULTIPOLYGON (((-96.78312 33.07242, -96.783428...","MULTIPOLYGON (((-96.76707 33.07242, -96.767686...",POINT (-96.79917 33.07242)
2,7496,-96.73341,33.15334,"MULTIPOLYGON (((-96.71736 33.15334, -96.717668...","MULTIPOLYGON (((-96.70131 33.15334, -96.701926...",POINT (-96.73341 33.15334)
3,1119,-96.83614,33.11117,"MULTIPOLYGON (((-96.82009 33.11117, -96.820398...","MULTIPOLYGON (((-96.80404 33.11117, -96.804656...",POINT (-96.83614 33.11117)
4,1249,-96.84477,33.13475,"MULTIPOLYGON (((-96.82871999999999 33.13475, -...","MULTIPOLYGON (((-96.81267 33.13475, -96.813286...",POINT (-96.84477 33.13475)
5,3789,-96.74276,33.10600,"MULTIPOLYGON (((-96.72671 33.106, -96.72701839...","MULTIPOLYGON (((-96.71066 33.106, -96.71127679...",POINT (-96.74276 33.106)
6,151,-96.82622,33.11653,"MULTIPOLYGON (((-96.81017 33.11653, -96.810478...","MULTIPOLYGON (((-96.79412 33.11653, -96.794736...",POINT (-96.82622 33.11653)
7,6202,-96.90773,33.16028,"MULTIPOLYGON (((-96.89168 33.16028, -96.891988...","MULTIPOLYGON (((-96.87563 33.16028, -96.876246...",POINT (-96.90773 33.16028)
8,6144,-96.75194,33.20155,"MULTIPOLYGON (((-96.73589 33.20155, -96.736198...","MULTIPOLYGON (((-96.71984 33.20155, -96.720456...",POINT (-96.75194 33.20155)
9,2510,-96.69223,33.14326,"MULTIPOLYGON (((-96.67617999999999 33.14326, -...","MULTIPOLYGON (((-96.66013 33.14326, -96.660746...",POINT (-96.69223 33.14326)


In [40]:
import folium

# Create a map centered around the average latitude and longitude
map_center = [df['LATITUDE'].mean(), df['LONGITUDE'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12, tiles="cartodb-darkmatter")

# Add store locations as markers
for _, row in gpd_df.iterrows():
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=f"Store ID: {row['STORE_ID']}"
    ).add_to(mymap)

# Add delivery radius polygons
for _, row in gpd_df.iterrows():
    folium.GeoJson(
        row['STORE_LOCATION_1MILE_DELIVERY_RADIUS'],
        name=f"1 Mile Radius - Store {row['STORE_ID']}",
        style_function=lambda x: {'color': 'green', 'weight': 1, 'fillOpacity': 0.2}
    ).add_to(mymap)
    folium.GeoJson(
        row['STORE_LOCATION_2MILE_DELIVERY_RADIUS'],
        name=f"2 Mile Radius - Store {row['STORE_ID']}",
        style_function=lambda x: {'color': 'orange', 'weight': 1, 'fillOpacity': 0.2}
    ).add_to(mymap)

# Display the map
mymap